# Tutorial: Building a Code Review Assistant with MCP in Agent Spec

*In this tutorial, we will create a powerful code review assistant using Agent Spec and MCP to automate pull request reviews, enhancing code quality with AI-driven insights.*

![image](https://miro.medium.com/v2/resize:fit:600/format:webp/1*_LJLsR-EL1Ua_vXIAuBLxg.png)

## Introduction

In today’s fast-paced development environment, code reviews are crucial but can be time-consuming. This tutorial is designed for individuals and teams who want to leverage the power of AI agents to automate the review of pull requests. Whether you’re working on open-source projects or enterprise software, integrating AI-powered code review tools can save time, reduce errors, and ensure coding standards are consistently enforced.

The goal of this guide is to walk you through the process of building a **Code Review Assistant** using Agent Spec, a framework-agnostic specification for portable AI agents and MCP (Model Context Protocol), which enables seamless interaction with tools and external systems. By the end of this tutorial, you’ll have a functional assistant capable of analyzing code changes and providing detailed feedback with actionable insights.

You’ll also discover how to export and reuse your assistant across multiple compatible agentic frameworks, giving you ultimate flexibility.

## Why Agent Spec?

**Agent Spec** is a framework-agnostic declarative specification designed to make AI agents and workflows portable, reusable, and executable across any compatible framework.

**Agent Spec** addresses the ongoing challenge of fragmentation in AI agent development, where different frameworks like AutoGen, LangGraph, and CrewAI lack a shared foundation, hindering portability and reuse.

**Agent Spec** enables developers to define AI agents and workflows that can be easily run, maintained, and adapted across various platforms and runtimes without rewriting code. This standardization benefits enterprises by streamlining scalability, reducing platform lock-in, simplifying maintenance, and enabling seamless collaboration both internally and with external partners.

You can find more about the Agent Spec motivation and vision at https://oracle.github.io/agent-spec/agentspec/intro_and_motivation.html

Also, you can read the full Agent Spec documentation at https://oracle.github.io/agent-spec/

We will use WayFlow (see https://github.com/oracle/wayflow) and LangGraph (see https://github.com/langchain-ai/langgraph) as the runtimes in this tutorial.

## Tutorial Overview: What we are building

![image2](https://miro.medium.com/v2/resize:fit:600/format:webp/1*nXUzkrS2bcp144EEwZPx_g.png)

In this tutorial, we are building a Code Assistant specifically designed for reviewing GitHub pull requests. The assistant will be equipped with tools to interact with GitHub, analyze code changes, and provide detailed feedback to improve code quality.

The agent leverages MCP (Model Context Protocol) tools to connect with GitHub, allowing it to list pull requests, read their contents, and retrieve file details for comprehensive reviews.

![image4](https://miro.medium.com/v2/resize:fit:600/format:webp/1*P4n7o5dOCXU4TdJuTQ10Xg.png)

[MCP](https://modelcontextprotocol.io/docs/getting-started/intro) is a standardized framework that streamlines communication between AI agents and external systems through a consistent and secure interface. It provides a suite of tools and protocols for agents to perform various tasks, access multiple data sources, and integrate with different platforms seamlessly. 

By simplifying the process of connecting with disparate systems, MCP allows developers to focus on building core agent functionality without getting bogged down by the details of each integration. This results in greater flexibility and scalability for AI-based solutions.

Let’s get started!

## Step 1. Setting up the environment

First, let's create and activate a virtual environment (outside of this Notebook):

```bash
# Create and activate a virtual environment (run in your terminal)
python -m venv venv-agentspec 
source venv-agentspec/bin/activate # On Windows: venv-agentspec\Scripts\activate
```

Next, let’s install pyagentspec, wayflowcore which are the packages we need to run this tutorial.

```bash
pip install "pyagentspec[langgraph_mcp]@git+https://github.com/oracle/agent-spec@main#subdirectory=pyagentspec"
pip install "wayflowcore@git+https://github.com/oracle/wayflow@main#subdirectory=wayflowcore"
```

You can also clone the repositories to stay up to date with the latest features ([agent-spec](https://github.com/oracle/agent-spec), [wayflow](https://github.com/oracle/wayflow)).

## Step 2. Configuring your LLM

WayFlow supports several LLM API providers. For an overview of supported LLMs, see the guide [How to Use LLMs from Different Providers](https://oracle.github.io/wayflow/core/howtoguides/llm_from_different_providers.html).

Start by selecting an LLM from one of the available providers:

You can use model providers such as OpenAI:

In [ ]:
from pyagentspec.llms import OpenAiConfig

llm_config = OpenAiConfig(
    name="openai-llm",
    model_id="model-id", # e.g. "gpt-4.1"
)

And you can also use locally models, e.g. with Ollama:

In [ ]:
from pyagentspec.llms import OllamaConfig

llm_config = OllamaConfig(
    name="ollama-llm",
    model_id="model-id", # e.g. gpt-oss:20b
    url="url/to/ollama_model" # e.g. localhost:11434
)

## Step 3. Setting up MCP Tools for the Agent

MCP allows agents to interact with external services through a standardized client transport. In this tutorial, we’ll set up tools to interact with GitHub for reviewing pull requests.

Agent Spec supports integration with MCP servers to provide tools dynamically. We’ll configure an MCP client to connect to the GitHub’s MCP server:

In [ ]:
from pyagentspec.mcp.clienttransport import StreamableHTTPTransport 
from pyagentspec.mcp.tools import MCPToolBox 

mcp_server_url = "https://api.githubcopilot.com/mcp/" 
headers = {"Authorization": f"Bearer {'GITHUB_PERSONAL_ACCESS_TOKEN'}"}
mcp_client = StreamableHTTPTransport(
    name="mcp_client", url=mcp_server_url, headers=headers
)
mcp_toolbox = MCPToolBox(
    name="mcp_toolbox", 
    client_transport=mcp_client, 
    tool_filter=[
        "list_pull_requests", "pull_request_read", "get_file_contents"
    ]
)

Here, the `MCPToolBox` is configured with specific tools like `list_pull_requests`, `pull_request_read` and `get_file_contents` to interact with GitHub pull requests and repository contents. To manage context length and avoid overwhelming the agent with excessive data, we use only the tools that are necessary for the task at hand. For example, we use `list_pull_requests` to see available PRs, `pull_request_read` to review a specific PR, and `get_file_contents` to access relevant files. This focused approach helps keep the context concise and relevant.

To discover available tools and their names, refer to the MCP server documentation or the specific service integration guide you’re using (e.g., [GitHub MCP server documentation](https://github.com/github/github-mcp-server)). These resources will list the tools provided by the server, their purposes, and any required parameters, ensuring you select the most appropriate tools for your agent’s tasks.

**Note:** To obtain a GitHub Personal Access Token, visit https://github.com/settings/tokens, click on “Generate new token”, select the necessary permissions (e.g., repo access for pull requests), and copy the generated token. Replace ‘GITHUB_PERSONAL_ACCESS_TOKEN’ with your actual token in the code above. Ensure you keep this token secure and do not commit it to version control.

## Step 4. Writing the Instructions for the Agent
The instructions (also known as the system prompt) define the agent’s behavior. Here, the instructions describe the agent’s persona (Code Reviewer), the workflow to follow for reviewing pull requests, and the expected output format.


In [ ]:
CUSTOM_INSTRUCTIONS = """ 
 You are an experienced code reviewer analyzing GitHub pull requests. Your primary goal is to understand proposed changes and deliver thorough, constructive feedback that enhances code quality and project alignment.
**Workflow:** 
 - **1. PR Overview:** Begin by gathering high-level details about the pull request. 
 - **2. Changed Files & Diffs:** Retrieve and review all modified files and code diffs. 
 - **3. Context Gathering:** Load any additional related files as needed (tests, configs). Review repository structure for context if necessary. 
 - **4. Code Analysis:** Examine the code for correctness, maintainability, standards compliance, performance, security, and architectural impact. 
 - **5. Inline Comments:** Provide specific, actionable comments on significant code changes, including suggestions for improvement where relevant. 
 - **6. Summary Assessment:** Conclude with a clear summary, using this structure: 
 i. **Description:** What the PR does. 
 ii. **Strengths:** What's done well. 
 iii. **Opportunities for Improvement:** Areas to address. 
 iv. **Additional Suggestions:** Any further feedback.
**Guidelines:** 
- Use Markdown. Use bullet points for everything.
- Include code snippets when suggesting changes.
- Complete all steps before responding.
- Keep feedback concise, specific, and actionable.
- Use all available tools to support your analysis. Most common usage for reference:
    - `list_pull_requests` when the user does not specify a PR link.
    - `pull_request_read` when the user gives a PR URL.
    - `get_file_contents` if you need to inspect related files beyond the diff.
Your review should help authors deliver higher-quality code and foster continuous improvement. 
 """.strip()

**Tip:** Debugging LLM Prompts: Passing a confusing prompt to an Agent may lead to unexpected behavior. If you receive an unexpected answer from the agent, try to directly ask the agent, “What led you to answer? Is there anything confusing in the given set of instructions?”

Very often, LLMs will point to what is potentially confusing.

## Step 5. Assembling the Agent
Now that the tools and instructions are ready, we can create the agent:

In [ ]:
from pyagentspec.agent import Agent
agent_config = Agent( 
   name="github_agent", 
   llm_config=llm_config, 
   system_prompt=CUSTOM_INSTRUCTIONS, 
   toolboxes=[mcp_toolbox] 
)

## Step 6. Exporting the Agent to Agent Spec
We can now export the Agent Spec configuration. This configuration is framework agnostic, which means that you can author the assistant once and then re-use it with any Agent Spec compatible framework.


In [ ]:
from pyagentspec.serialization import AgentSpecSerializer
serialized_agent = AgentSpecSerializer().to_json(agent_config)

You can find the Agent Spec configuration at https://github.com/oracle/agent-spec.

## Step 7. Running the Agent

We will now run the agent we defined above using multiple runtimes like WayFlow and LangGraph. This demonstrates the power of Agent Spec’s framework-agnostic design, allowing your assistant to operate across different environments and take advantage of each runtime’s unique strengths and optimizations for broader compatibility and flexibility.

### A. Using WayFlow

Let’s run this agent using WayFlow:

In [ ]:
from wayflowcore.agentspec import AgentSpecLoader as WayFlowLoader 
from wayflowcore.agent import Agent as RuntimeAgent 
from wayflowcore.mcp import enable_mcp_without_auth

enable_mcp_without_auth() 
agent: RuntimeAgent = WayFlowLoader().load_json(serialized_agent)

Note: Although `enable_mcp_without_auth()` is used, authentication still occurs through the Personal Access Token (PAT) described above. Ensure your PAT is securely stored and has the appropriate read/write permissions.

Now, we have instantiated theagent. We can run it as:

In [ ]:
conversation = agent.start_conversation() 
pr_link = "<PR LINK>" 
user_query = f"Review changes of {pr_link}"
conversation.append_user_message(user_query) 
status = await conversation.execute_async()

**Note:** Replace <PR LINK> with the actual URL of the PR.

### B. Using LangGraph

We can also run it using LangGraph:

In [ ]:
import asyncio
from pyagentspec.adapters.langgraph import AgentSpecLoader

langgraph_assistant = AgentSpecLoader().load_json(serialized_agent)
config = {"configurable": {"thread_id": "1"}}

pr_link = "<PR LINK>"
user_query = f"Review changes of {pr_link}"

result = await langgraph_assistant.ainvoke(
    {"messages": [{"role": "user", "content": user_query}]},
    config,
)


**Note:** Replace <PR LINK> with the actual URL of the PR.

Here is what the result may look like for both:

> I've reviewed the pull request at <PR LINK>. Below is my detailed feedback:
> - **Description:** This PR introduces enhancements to pretty printing. 
>  - **Strengths:** The code is well-structured, with clear documentation and logical organization. 
>  - **Opportunities for Improvement:** 
>    - Consider adding unit tests for the new configuration options to ensure robustness. 
>    - In `print.py`, line 45, the error handling for invalid inputs could be more comprehensive. 
>  - **Additional Suggestions:** It might be beneficial to include a changelog entry for visibility.

Let me know if you'd like me to dive deeper into any specific file or aspect of this PR.

👏 Congratulations, you have now built your first code assistant agent with Agent Spec and ran it using WayFlow and LangGraph!

By now, you should have:

- A Python script that defines a GitHub code review agent using Agent Spec.
- An MCP client configured to talk to the GitHub MCP server with authentication.
- The same agent running on WayFlow, via `WayFlowLoader.load_json(serialized_agent)` and Langgraph using `AgentSpecLoader.load_json(serialized_agent)`.
- An AI reviewer that can read PRs and produce structured feedback.

## Common Errors

When executing the agent, you might encounter several common errors. To troubleshoot effectively, enable detailed debug logs by adding the following at the beginning of your script:

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

This will provide comprehensive logs to help identify issues. Below are some common errors and their potential causes or solutions:

1. **Incorrect MCP Server URL:** If the URL to the MCP server is wrong, you will see a log entry indicating “HTTP/1.1 404 Not Found” for the specified URL. Double-check the server URL in your StreamableHTTPTransport configuration and ensure it matches the documented endpoint.

2. **Invalid Personal Access Token (PAT):** An error like “Client error ‘400 Bad Request’ for url ‘https://api.githubcopilot.com/mcp'” typically indicates an incorrect PAT or insufficient permissions. Verify that your GitHub PAT is correct and has the necessary scopes and permissions. Regenerate a new token if needed at https://github.com/settings/tokens.

3. **Incorrect Tool Name in Toolbox:** If a tool name specified in the MCPToolBox is wrong, you will encounter a `NoSuchToolFoundOnMCPServerError` in WayFlow. Review the tool names in your `tool_filter` list against the MCP server documentation (e.g., GitHub MCP server docs) to ensure they match exactly (e.g., `list_pull_requests`, not `list_prs`).

4. **LLM Configuration Issues:** If your LLM fails to initialize, you might see errors related to invalid model IDs, API keys, or connection issues. For instance, with Ollama, a “ConnectionRefused” error indicates the local server URL might be incorrect or the server is not running. Verify your llm_config settings and ensure the model provider’s service is accessible.

If you encounter an error not listed here, consult the debug logs for detailed stack traces and search for relevant error codes in the Agent Spec, WayFlow, or LangGraph documentation. Community forums and GitHub issues for these projects can also provide additional troubleshooting insights.

## Going Further

Now that you’ve built a basic Code Review Assistant, there are several ways to enhance its capabilities. Here are some concise suggestions to take your agent further:

1. **Enable Commenting on Pull Requests:** Extend your agent to post comments on GitHub by adding the `pull_request_review_write` tool to your `MCPToolBox`. Update instructions to include posting inline feedback, integrating reviews into the development workflow.

2. **Summarize Code Changes:** Add a workflow step to summarize code diffs before detailed review to manage context length. Process diffs from `pull_request_read` to create concise summaries for the agent to reference.

3. **Support Multiple Repositories:** Enable handling of pull requests across multiple repositories. Adjust instructions to switch contexts based on user queries, possibly using dynamic tool setups or additional MCP tools.

4. **Add Custom Review Criteria:** Tailor reviews with project-specific criteria by updating the system prompt with guidelines for patterns, naming, or performance. This ensures feedback matches team and project standards.

These enhancements can greatly improve your Code Review Assistant’s functionality. Experiment with these ideas to adapt the agent to your needs.

## GitHub Repository

You can access all the code used in this article on the [AgentSpec GitHub](https://github.com/oracle/agent-spec/blob/main/docs/pyagentspec/source/code_examples/ext_code_assistant_tutorial.py).

## Conclusion

In this guide, you learned how to build a simple yet powerful Code Assistant Agent for GitHub pull request reviews by equipping an Agent with MCP tools using Agent Spec and ran it with Wayflow and LangGraph.